# Natural language processing
# As supervised problem

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,rattle,actually,mournful,thither,vanish,globe,twentieth,mistake,eyed,reel,...,wonderland,shin,previously,season,seat,squeaking,she,say,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## BoW with Random Forest

In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.987147335423

Test set score: 0.889097744361


## BoW with Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.957993730408

Test set score: 0.915883458647


## BoW with Gradient Boosting

In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.884639498433

Test set score: 0.873120300752


## Same model, new inputs

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [13]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [14]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [15]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.697613721104


col_0,Austen,Carroll
row_0,,
Austen,1564,105
Carroll,706,307


## Challenge 0

Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to improve performance. Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires. Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.

### Creating Training and Test Sets 

In [16]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

### Using SVM 

In [18]:
from  sklearn.svm  import LinearSVC

svc = LinearSVC()
svc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('\nTraining set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.984326018809

Test set score: 0.898026315789


### Using Random Forest with 100 estimators, max depth 20 and max_features "auto"

In [22]:
rfc1 = ensemble.RandomForestClassifier(max_depth=20,max_features='auto', n_estimators=100)

rfc1.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('\nTraining set score:', rfc1.score(X_train, y_train))
print('\nTest set score:', rfc1.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.858307210031

Test set score: 0.839755639098


### Using Random Forest with 200 estimators, max depth 50 and max_features "auto"

In [23]:
rfc2 = ensemble.RandomForestClassifier(max_depth=50,max_features='auto', n_estimators=200)

rfc2.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('\nTraining set score:', rfc2.score(X_train, y_train))
print('\nTest set score:', rfc2.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.920062695925

Test set score: 0.890037593985


### Gradient Boosting  with 50 iterations, use 10-deep trees, and set our loss function to deviance

In [52]:
params = {'n_estimators': 50,
          'max_depth': 10,
          'loss': 'deviance'}

# Initialize and fit the model.
clf1 = ensemble.GradientBoostingClassifier(**params)
clf1.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('\nTraining set score:', clf1.score(X_train, y_train))
print('\nTest set score:', clf1.score(X_test, y_test))

(3190, 3062) (3190,)

Training set score: 0.912852664577

Test set score: 0.882988721805


### Gradient Boosting  with 100 iterations, use 20-deep trees, and set our loss function to deviance

In [26]:
params = {'n_estimators': 100,
          'max_depth': 20,
          'loss': 'deviance'}

# Initialize and fit the model.
clf2 = ensemble.GradientBoostingClassifier(**params)
clf2.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('\nTraining set score:', clf2.score(X_train, y_train))
print('\nTest set score:', clf2.score(X_test, y_test))

(3190, 3062) (3190,)

Training set score: 0.978369905956

Test set score: 0.895676691729


## Challenge 1

Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.



In [28]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [40]:
# Clean the poems data.
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
macbeth = re.sub(r'VOLUME \w+', '', macbeth)
macbeth = re.sub(r'CHAPTER \w+', '', macbeth)
macbeth = text_cleaner(macbeth)
print(macbeth[:100])

Actus Primus. Scoena Prima. Thunder and Lightning. Enter three Witches. 1. When shall we three meet 


In [41]:
# Parse our cleaned data.
macbeth_doc = nlp(macbeth)

In [42]:
# Group into sentences.
macbeth_sents = [[sent, "Shakespeare"] for sent in macbeth_doc.sents]
macbeth_sents = stories_sents[0:len(alice_sents)]

In [43]:
# Build a new Bag of Words data frame for Poems word counts.
# We'll use the same common words from Alice and Persuasion.
macbeth_sentences = pd.DataFrame(macbeth_sents)
macbeth_bow = bow_features(macbeth_sentences, common_words)

print('done')

Processing row 0
done


In [47]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Macbeth_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    macbeth_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Macbeth_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Shakespeare'] * macbeth_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Macbeth_test, y_Macbeth_test))
lr_Macbeth_predicted = lr.predict(X_Macbeth_test)
pd.crosstab(y_Macbeth_test, lr_Macbeth_predicted)


Test set score: 0.203176704169


col_0,Austen,Carroll
row_0,,
Carroll,706,307
Shakespeare,384,114


In [54]:
print('svc score: ', svc.score(X_Macbeth_test, y_Macbeth_test))

print('\nrfc1 score: ', rfc1.score(X_Macbeth_test, y_Macbeth_test))
print('\nrfc2 score: ', rfc2.score(X_Macbeth_test, y_Macbeth_test))


print('\nclf1 score: ', clf1.score(X_Macbeth_test, y_Macbeth_test))
print('\nclf2 score: ', clf2.score(X_Macbeth_test, y_Macbeth_test))

svc score:  0.225016545334

rfc1 score:  0.124420913302

rfc2 score:  0.168100595632

clf1 score:  0.167438782263

clf2 score:  0.217074784911
